In [1]:
import sys
import os
import cv2
from pathlib import Path
from PIL import Image, ImageSequence
import numpy as np
from ultralytics import YOLO

In [2]:
# Find pyproject.toml in parent directories and add its directory to sys.path
current_path = Path.cwd()
for parent in [current_path] + list(current_path.parents):
    pyproject_file = parent / "pyproject.toml"
    if pyproject_file.exists():
        sys.path.insert(0, str(parent))
        break

In [3]:
# Create output directory and save result images
output_dir = current_path.parent / "output"
output_dir.mkdir(exist_ok=True)

In [4]:
# Load a pre-trained YOLO model (e.g., YOLOv8s for small, yolov8n for nano)
model = YOLO("yolo12l.pt") 

In [5]:
gif_path: Path = Path("../images/rabbits.gif")

In [6]:
# Open GIF and iterate frames
im = Image.open(gif_path)


In [7]:
frames_out = []
durations = []

In [8]:
def shrink_image(arr_input, max_dim=320):
    h, w = arr_input.shape[:2]
    if max(h, w) > max_dim:
        scale = max_dim / max(h, w)
        new_w, new_h = int(w * scale), int(h * scale)
        arr_output = cv2.resize(arr_input, (new_w, new_h), interpolation=cv2.INTER_AREA)
    else:
        arr_output = arr_input
    return arr_output

In [9]:
for frame in ImageSequence.Iterator(im):
    durations.append(frame.info.get("duration", 100))  # ms per frame (default 100)
    rgb = frame.convert("RGB")
    arr = np.array(rgb)

    # shrink image if larger than max dimension
    arr = shrink_image(arr, max_dim=320)

    # Run YOLO prediction on the frame (array in RGB)
    results = model(arr)
    # Get annotated image (numpy array)
    annotated = results[0].plot()

    # Convert annotated array back to PIL Image and collect
    frames_out.append(Image.fromarray(annotated))



0: 640x384 1 bowl, 60.6ms
Speed: 2.1ms preprocess, 60.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 1 bowl, 36.5ms
Speed: 1.2ms preprocess, 36.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 1 bowl, 35.0ms
Speed: 1.3ms preprocess, 35.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 sheeps, 1 bowl, 34.8ms
Speed: 1.3ms preprocess, 34.8ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 35.2ms
Speed: 1.2ms preprocess, 35.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bowl, 34.2ms
Speed: 1.2ms preprocess, 34.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 sheep, 1 bowl, 31.5ms
Speed: 1.2ms preprocess, 31.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bowl, 31.4ms
Speed: 1.1ms preprocess, 31.4ms inference, 1.3ms postprocess per im

In [10]:
# Save annotated frames back to a GIF
if frames_out:
    out_path = output_dir / f"{gif_path.stem}_predicted.gif"
    frames_out[0].save(
        out_path,
        save_all=True,
        append_images=frames_out[1:],
        duration=durations,
        loop=0,
        optimize=False,
    )
    print("Saved predicted GIF to:", out_path)
else:
    print("No frames extracted from:", gif_path)

Saved predicted GIF to: /home/masayuki/Documents/Projects/Python/YOLO_camera/output/rabbits_predicted.gif
